In [ ]:
%matplotlib widget
from IPython.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))

# Guidance Bits for Digital Annealer

By default Digital Annealer (DA) and the Emulator carry out the optimization by starting from a random state in each run and try to reach the optimum energy value of decision variables of QUBO. However, it can be helpful in starting the annealing algorithm not from a random initialization (or e.g. all-zero or all-one assignment), but instead from a initial good _guess_ of the solution. It can be done by passing Digital Annealer parameter named _guidance_config_. _guidance_config_ is a dictionary where key represents polynomial index and value of that key is a boolean value. If the boolean value is $1$ or $True$ then the polynomial index of the QUBO is part of configuration else it will be $0$ or $False$. In this _guidance_config_ is defined and from this configuration Digital Annealer (DA) or Emulator will start the annealing process.

User does not need to define _guidance_config_ from scratch. There are helper functions availble in dadk.Binpol to define _guidance_config_ and pass it to DA. _dadk.BinPol.GuidanceConfig_ produces a dictionary that can be used to initialize bits for the annealing algorithm. It accepts input structure of QUBO which is defined by object of _dadk.BinPol.VarShapeSet_ . Once the QUBO is defined one can use its _VarShapeSet_ object to define _GuidanceConfig_ object. Beforehand we know the configuration/solution from where the annealing should start, respective polynomial indexes of configuration/solution in _GuidanceConfig_ object can be set _True_ or _False_ using set_bit method. If the polynomial indexes are slack bits then _set_slacks_ method can be used. In this way guidance bits can be defined using _dadk.BinPol.GuidanceConfig_.

In this following example _guidance_config_ is the object of _GuidanceConfig_ and using _set_bit_ method the configuration /solution is defined.

### Creation of the QUBO

For guidance bits experiment travelling sales man problem is chosen. Where $x_{t,c}$ means at time $t$, city $c$ is visited. $c \in (0,1,2,.., N-1)$ and $t \in (0,1,2,.., N-1)$

A term to ensure that at a given moment the travelling salesman is visiting only one city. 
$$
P_{\text{time}} = f_{\text{rules}} * \sum_{t=0}^{N - 1} \left(1 - \sum_{c=0}^{N - 1} x_{t,c} \right)^2
$$

Another term for ensuring that every customer is visited only once.
$$
P_{\text{city}} = f_{\text{rules}} * \sum_{c=0}^{N - 1} \left(1 - \sum_{t=0}^{N - 1} x_{t,c} \right)^2
$$

The last term is the objective function. If the salesman goes through an edge, then we add up the cost involved in travelling that path. All the costs are then added up.
$$
P_{\text{distance}} = f_{\text{distance}} *  \sum_{t=0}^{N - 1} \sum_{c_0=0 \\c_1=0 \\c_0 \ne c_1}^{N - 1} \Delta_{c_0, c_1} x_{t,c_0} x_{(t+1) \% N, c_1} 
$$

$factor\_rules$, $factor\_distance$ are the penalty factors for the polynomials.
$$
    HQ = factor\_rules*P_{\text{time}} + factor\_rules*P_{\text{city}} + factor\_distance*P_{\text{distance}}
$$

### Usage

This notebook uses optimizer framework to implement the problem and visualize the results. Demo_Guidance_Model class code is where problem is formulated and using optimizer framework. Please run the code section. It will display the optimizer workflow. The main part implemented in the class Demo_Guidance_Model inheriting from OptimizerModel. Provided as argument to an Optimizer object upon initialization which creates a dashboard/GUI environment to run models, test and tune. With this frame work different workflows can be implemented. Used sections and use is explained below.

#### Setup scenario

In this section the input parameters for the problem is defined which corresponds to the _load_ method of Demo_Guidance_Model class. Number of cities in the problem is defined by size parameter. To use the guidance bits one can set _use guidance_ parameter True else False. Then please click call to formulate problem. The guidance solution is displayed with distance cost and visualization. Also the _guidance_config_ is  defined and it is the object of GuidanceConfig class. Guidance solution is configured in _guidance_config_. _get_dict_ method on the object _guidance_config_ will return the required dictionary of configuration. This dictionary will be passed as a parameter to Digital Annealer and defined in method _set_calculated_solver_parameter_ of the class _Optimizer_. 

#### Build QUBO

For the problem prepared the QUBO is built once clicked on call. One can change the penalty factors here. It is advisable to accept the defaults.

#### Solve annealing

Annealing parameters can be defined here. Please accept the defaults and click call. After optimization the solution energy of the QUBO will be displayed along with the solution.

#### Anneal tracker

All the timings are logged and displayed graphically here. Also other graphics related to Emulator insights will be displayed.Following is the graphics without use of guidance bits. In this case annealing for each run has been started from a random configuration. Normally such configuration has a high energy values. If we take a look at energy log in the following figure then we can see we are stating annealing with a configuration with high energy, at the starting phase the temperature is high and therefore the search agent can overcome any valley just using the high temperatures. As a result of high initial temperature agent can jump from one search space region to other easily and this is the global search of optimum of the agent. As the temperature decreases the agent searches in the local area for optimum and cannot overcome energy valley just with temperature and then we add offset energies to overcome valley and continue the search for optimum. This results in the local search. In this way agent do the global and local search which can be seen in energy logs and waiting cycle graphics in the following graphics. 
![no_guidance_graphic](figures/no_guidance_graphic.PNG)

###### Graphics with guidance bits 

If the guidance configuration is close to optimum or good local minimum as also in this case, then annealing process starts from very good low energy configuration. Initially because of high temperatures we will jump out of the good, low energy solution minima and start to do a global search jumping from one region to the other and we will go away from intended area of solution search as can be seen in the following energy log graphics. Finally we will do local search at low temperatures in some other area. Using the guidance bits for Digital Annealer this aspect must be taken into consideration. As we can see in the following graphics that guidance bits is the solution and annealing could not find it again.   
![guidance_graphic](figures/guidance_graphic.PNG)

###### Graphics with guidance bits and low temperatures
If we keep the start and end temperatures very low then annealing process will only search the local space search as can be seen in the following graphic. All these things must be considered while using guidance bits. With bad chosen guidance bits the risk exists to stuck in a bad local minima. One should know that guidance bits can be useful if it is expected that a global/good solution is nearby. Otherwise guidance bits can lead to stuck somewhere in the solution space and never get out of it
![guidance_graphic](figures/guidance_graphic_low_temp.PNG)


#### Report
In this section the bit plot of the solution can be seen. Red spot on the bit plot indicates at what time the salesman is visiting which city.


#### Visualization

In this section we are visualizing the guidance solution tour and Digital Annealer/Emulator tour.

In [ ]:
import matplotlib.pyplot as plt

from PIL import Image, ImageDraw, ImageFont

from math import sqrt, sin, cos, pi, floor, ceil
from random import seed, uniform, randint, random

from dadk.Optimizer import *

import networkx as nx

class Demo_Guidance_Model(OptimizerModel):
    
    ##################################################################################################################

    def __init__(self, persistent_file="M_04_Guidance.dao"):
        OptimizerModel.__init__(
            self,
            name = 'Demo_Guidance',
            load_parameter=[
                {'name':'N', 'type':'int_slider', 'value':12, 'min':2, 'max': 90, 
                 'label':'Size', 'description':'problem size'},
                {'name': 'random_seed', 'type': 'int_slider', 'value': 42, 'min': 0, 'max': 9999,
                 'label': 'Random seed', 'description': 'Seed for random number generation'},
                {'name':'use_guidance', 'value':True, 'type':'check', 'options':[True, False],
                 'label':'Use guidance', 'description':'Use nearest neighbour for guidance'}
            ],            
            build_qubo_parameter=[
                {'name':'factor_rules', 'type':'float_slider', 'value':100.0, 'min':0.0, 'max':10**6,
                 'label':'factor rules'},
                {'name':'factor_distance', 'type':'float_slider', 'value':1.0, 'min':0.0, 'max':10**3,
                 'label':'factor distance'}
            ], 
            #
            # Define default values for the "Solve annealing" tab.
            # The user can modify these fields, or press the button 
            # "Default" and get these values again.
            #
            default_solver_parameter={
                'number_iterations':20000,
                'number_iterations_CPU':10000,
                'temperature_end':0.1,
                'offset_increase_rate':2.0,
                'temperature_start': 50000,
                'processor': 'CPU',
                'bit_precision':32,
                'graphics':True
            },
            persistent_file=persistent_file
        )
    
    ##################################################################################################################
    def plot_path(self, path=[], title="???", color="red"):
    ##################################################################################################################

        self.graph = nx.Graph()

        nodes={}
        for i in range(self.N):
            nodes[i] = str(i);
            self.graph.add_node(nodes[i], pos=(self.cities[i][0], self.cities[i][1]))

        for i in range(self.N):
            for j in range(i+1, self.N):
                if [i, j] in path or [j, i] in path: 
                    self.graph.add_edge(nodes[min(i,j)], nodes[max(i,j)], weight=10)
                else:
                    self.graph.add_edge(nodes[min(i,j)], nodes[max(i,j)], weight=0)

        fig = plt.figure(num=title, figsize=(8.0, 8.0))

        pos=nx.get_node_attributes(self.graph,'pos')

        # nodes
        nx.draw_networkx_nodes(self.graph, pos, nodelist=self.graph.nodes, node_size=100, node_color="red")

        # edges        
        edges_bold = [(u, v) for (u, v, d) in self.graph.edges(data=True) if d['weight'] == +10]
        nx.draw_networkx_edges(self.graph, pos, edgelist=edges_bold, width=2, edge_color=color)

        edges_thin = [(u, v) for (u, v, d) in self.graph.edges(data=True) if d['weight'] == 0]
        nx.draw_networkx_edges(self.graph, pos, edgelist=edges_thin, width=1, alpha=0.05, edge_color="black")

        # labels
        nx.draw_networkx_labels(self.graph, pos, font_size=6, font_family='sans-serif')

        plt.axis('off')
        plt.show()
            
    ##################################################################################################################
    def load(self, N=12, random_seed=42, use_guidance=True, silent=False):
    ##################################################################################################################
    
        self.N = N
        self.use_guidance = use_guidance
        
        self.guidance_route = None
        self.guidance_distance = None
                        
        DEBUG = False
       
        seed(random_seed)
        
        self.cities = []
        while len(self.cities) < self.N:
            x = round(uniform(-100, +100), 3)
            y = round(uniform(-100, +100), 3)
            if sqrt(x * x + y * y) <= 100:
                self.cities.append([x, y])
        
        self.distance_table = [[math.sqrt((self.cities[c_1][0] - self.cities[c_0][0])**2 + (self.cities[c_1][1] - self.cities[c_0][1])**2)
                                 for c_0 in range(N)] for c_1 in range(N)]
        
        if self.use_guidance:
            
            guidance_route_list = []            
            for start_index in range(self.N):
            
                free_cities = list(range(self.N))
                
                start_city = free_cities[start_index]
                guidance_route = [start_city]
                free_cities.remove(start_city)

                while len(free_cities) > 0:
                    last_city = guidance_route[-1]
                    next_city_list = [{"city":city, "distance":self.distance_table[last_city][city]} for city in free_cities]

                    def custom_sort(data):
                        return data["distance"]

                    next_city_list.sort(key=custom_sort)
                    next_city = next_city_list[0]["city"]
                    guidance_route.append(next_city)
                    free_cities.remove(next_city)
                    
                distance = sum(self.distance_table[guidance_route[t]][guidance_route[(t + 1) % self.N]] for t in range(self.N))  
                
                if DEBUG:
                    print("option : %s -> %.3f" % (guidance_route, distance))
                    path = [[guidance_route[t], guidance_route[(t + 1) % self.N]] for t in range(self.N)]  
                    self.plot_path(path=path, title="option = %.2f" % distance, color="red")
                
                guidance_route_list.append({"distance":distance, "guidance_route":guidance_route})
                
            def custom_sort(data):
                return data["distance"] 
            
            guidance_route_list.sort(key=custom_sort)
            
            self.guidance_route = guidance_route_list[0]["guidance_route"]
            self.guidance_distance = guidance_route_list[0]["distance"]
            
            print("guidance: %s -> %.3f" % (self.guidance_route, self.guidance_distance))
            
            path = [[self.guidance_route[t], self.guidance_route[(t + 1) % self.N]] for t in range(self.N)]  
            self.plot_path(path=path, title="Guidance = %.2f" % self.guidance_distance, color="blue")

    ##################################################################################################################
    def build_qubo(self, factor_distance, factor_rules, silent=False):
    ##################################################################################################################
        
        self.var_shape_set = VarShapeSet(BitArrayShape('x', (self.N, self.N), one_hot=OneHot.two_way, axis_names=['Time', 'City']))        
                        
        #--------------------------------------------------------------------------------

        self.P_time = BinPol(self.var_shape_set)   
        for t in range(self.N):
            self.P_time += (1 - BinPol.sum(Term(1, (('x', t, c),), self.var_shape_set) for c in range(self.N))) ** 2
        self.P_time *= factor_rules
        
        #--------------------------------------------------------------------------------

        self.P_city = BinPol(self.var_shape_set) 
        for c in range(self.N):
            self.P_city += (1 - BinPol.sum(Term(1, (('x', t, c),), self.var_shape_set) for t in range(self.N))) ** 2    
        self.P_city *= factor_rules        

        #--------------------------------------------------------------------------------
                        
        self.P_distance = BinPol(self.var_shape_set) 
        for t in range(self.N):
            self.P_distance += BinPol.sum(Term(self.distance_table[c_0][c_1], (('x', t, c_0), ('x', (t + 1) % self.N, c_1)), self.var_shape_set) 
                                   for c_0 in range(self.N) for c_1 in range(self.N) if c_0 != c_1) 
        self.P_distance *= factor_distance

        #--------------------------------------------------------------------------------

        self.HQ = self.P_time + self.P_city + self.P_distance
        
        #--------------------------------------------------------------------------------

        guidance_config = None
        if self.use_guidance:
            
            guidance_config = PartialConfig(self.var_shape_set)             
            for t in range(self.N):
                for c in range(self.N):
                    if self.guidance_route[t] == c:                                  
                        guidance_config.set_bit(('x', t, c), True)
                    else:
                        guidance_config.set_bit(('x', t, c), False)
            
        self.Optimizer.set_calculated_solver_parameter(guidance_config=guidance_config)
                
    ##################################################################################################################
    def prep_result(self, solution_list:SolutionList, silent=False):
    ##################################################################################################################

        configuration = solution_list.min_solution.configuration

        self.configuration = configuration
        self.start_table = None
        self.route = None
              
        if not silent:
            
            e_HQ = self.HQ.compute(self.configuration)
            e_time = self.P_time.compute(self.configuration)
            e_city = self.P_city.compute(self.configuration)
            e_distance = self.P_distance.compute(self.configuration)
            
            print("HQ         = %10.2F" % e_HQ)
            print("P_time     = %10.2F" % e_time)
            print("P_city     = %10.2F" % e_city)
            print("P_distance = %10.2F" % e_distance)
            print("")
            if e_time == 0 and e_city == 0:

                self.start_table = solution_list.min_solution.extract_bit_array('x')
                self.route = [min(c if self.start_table.data[t][c] else self.N for c in range(self.N)) 
                                  for t in range(self.N)]
                self.distance = sum(self.distance_table[self.route[t]][self.route[(t + 1) % self.N]] for t in range(self.N))

                print("+----+------------+----------+----------+")
                print("|  t | from -> to | distance |    total |")
                print("+----+------------+----------+----------+")
                total_distance = 0.0
                for t in range(self.N):
                    distance = self.distance_table[self.route[t]][self.route[(t + 1) % self.N]]
                    total_distance = total_distance + distance
                    print("| %2d | %4d -> %2d | %8.2F | %8.2F |" % (t, self.route[t], self.route[(t + 1) % self.N], distance, total_distance))
                print("+----+------------+----------+----------+")
            else:
                print("invalid solution, try again!")
            
    ##################################################################################################################
    def report(self, silent=False):
    ##################################################################################################################
        
        if self.start_table is not None:
            self.start_table.draw(figsize=(6.0, 6.0))
        
    ##################################################################################################################
    def draw(self):
    ##################################################################################################################
    
        if self.start_table is not None and self.route is not None:
            
            print("solution : %s -> %.3f" % (self.route, self.distance))
            solution_path = [[self.route[t], self.route[(t + 1) % self.N]] for t in range(self.N)]  
            self.plot_path(path=solution_path, title="Solution = %.2f" % self.distance, color="black")
            
            if self.use_guidance:
                print("guidance : %s -> %.3f" % (self.guidance_route, self.guidance_distance))
                guidance_path = [[self.guidance_route[t], self.guidance_route[(t + 1) % self.N]] for t in range(self.N)]
                self.plot_path(path=guidance_path, title="Guidance = %.2f" % self.guidance_distance, color="blue") 

            
optimizer = Optimizer(Demo_Guidance_Model(), auto_load=True)